# Скачивание постов из вк

In [140]:
import re
import vk_api
import random
import datetime
import time

In [146]:
def author_and_date_creation_extraction(authorship_string):
    
    creation_date_detection = ', (?P<date>[0-9][0-9][0-9][0-9])'
    muzz_splitter = '( muzz )|( музз )|( муzzz )|( с реверансом )|( посв\.? )'
    user_from_id_detection = r'\[(?P<user_id>((id)|(club))[0-9]+)\|(?P<user_name>[^\]]+)\]'
    user_from_link_detection = r'\[(?P<user_id>https:\/\/vk\.com\/[A-Za-z\._]+)\|(?P<user_name>[^\]]+)\]'
    
    match = re.search(creation_date_detection, authorship_string)
    if match != None:
        creation_date = int(match.group('date'))
    else:
        creation_date = None
    
    match = re.search(muzz_splitter, authorship_string)
    if match != None:
        start = match.span()[0]
        end = match.span()[1]
        authors = authorship_string[:start]
        #TODO в дальнейшем можно лемматизировать  вдохновителей (muzz), если они указаны через "посв."
        muzz_string = authorship_string[end:]
    else:
        authors = authorship_string
        muzz_string = None

    if re.search(user_from_id_detection, authors) != None:
        match_iter = re.finditer(user_from_id_detection, authors)
    else:
        match_iter = re.finditer(user_from_link_detection, authors)
    author_list = []
    for match in match_iter:
        author_id_or_link = match.group('user_id')
        author_name = match.group('user_name')
        author_list.append([author_id_or_link, author_name])

    return [author_list, creation_date, muzz_string]

In [2]:
with open('token.txt', encoding='utf-8') as file:
    token = file.read()

In [151]:
posts = []
for offset in range(100, 1001, 100):
    vk = vk_api.VkApi(token=token)
    vk_session = vk.get_api()
    poroshok = vk_session.wall.get(owner_id = -31481258, offset = offset, count = 100)
    for post in poroshok['items']:
        post_id = int(post['id'])
        date = datetime.datetime.fromtimestamp(int(post['date'])).strftime('%Y-%m-%d %H:%M:%S')
        text_author_list = post['text'].split('©')
        text = text_author_list[0]
        # TODO в дальнейшем можно добавить в базу данных вдохновителей (muzz) и тоже атрибутировать их к постам
        authors, creation_date, muzz_string = author_and_date_creation_extraction(text_author_list[1])
        posts.append([post_id, text, date, authors, creation_date])
    print(f"Finished first {offset} posts")
    time.sleep(random.uniform(2, 10))

Finished first 100 posts
Finished first 200 posts
Finished first 300 posts
Finished first 400 posts
Finished first 500 posts
Finished first 600 posts
Finished first 700 posts
Finished first 800 posts
Finished first 900 posts
Finished first 1000 posts


In [162]:
posts[2]

[107059,
 'олега очень возмущает\nсуществование глистов\nособенно во время строгих\nпостов\n',
 '2023-07-14 15:00:00',
 [['id934334', 'тры'], ['id135278152', 'Мурка']],
 None]

# Загрузка постов в БД (неактуальный вариант)

In [4]:
import sqlite3
import pandas as pd

In [10]:
con = sqlite3.connect('poroshki.db')
cur = con.cursor()

In [11]:
cur.execute('''DROP TABLE post;''')

In [12]:
cur.execute("""
CREATE TABLE post (
    id INT PRIMARY KEY, 
    text TEXT, 
    date TEXT,
    author_name TEXT
)
""")

In [13]:
cur.executemany("INSERT INTO post (id, text, date, author_name) VALUES (?, ?, ?, ?)", posts)

In [14]:
con.commit()
con.close()

In [ ]:
# if 'attachments' != []: не берём??

# post['text']
# post['date']
# link = f'https://vk.com/sandalporoshki?w=wall-31481258_{post['id']}'